TODO:
- need to locate the codon dictionaries from the main_package directory
- stop codon variable

In [1]:
# imports
import argparse
from Bio.SeqUtils import GC
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio import SeqIO
import math
import numpy as np
import pandas as pd
#import random
import main_package # my package


In [2]:
### SKIP until this is scripted
# parse arguments
parser = argparse.ArgumentParser()
parser.add_argument("protein", help="Name of protein sequence", type=str)
parser.add_argument("up", help="Up: 30bp 5' of sequence", type=str)
parser.add_argument("wt", help="WT: sequence of interest", type=str)
parser.add_argument("down", help="Down: 30bp 3' of sequence ", type=str)
parser.add_argument("--vect", help="Vect: sequence of interest in vector", type=str, default=False)
parser.add_argument("--codon_table", help="Specify codon table to use", type=str, default='Standard')
parser.add_argument("--homo_len", help="Length of homology arm in fwd primer", type=int, default=20)
parser.add_argument("--primer_len", help="Ideal max length of primers", type=int, default=60)
parser.add_argument("--melt_temp", help="Melting temp of fwd primer", type=int, default=50)
parser.add_argument("--rev_melt_temp", help="Melting temp of rev primer", type=int, default=55)
parser.add_argument("--syn_snp_rate", help="Percentage of synonymous SNPs 0-1", type=float, default=.05)
parser.add_argument("--stop_rate", help="Percentage of stop codon SNPs, default = keep 10% of stop SNPs", type=float, default=.10)
parser.add_argument("--rng_seed", help="Set seed for repoducibly selecting synonymous codon sites", type=int, default=42)
parser.add_argument("--out_dir", help="Local output directory e.g. 'data/'", type=str, default=.05)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--vect VECT] [--codon_table CODON_TABLE]
                             [--homo_len HOMO_LEN] [--primer_len PRIMER_LEN]
                             [--melt_temp MELT_TEMP]
                             [--rev_melt_temp REV_MELT_TEMP]
                             [--syn_snp SYN_SNP] [--out_dir OUT_DIR]
                             protein up wt down
ipykernel_launcher.py: error: the following arguments are required: up, wt, down


SystemExit: 2

/opt/anaconda3/envs/alignparse-environment/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
# generate missense codon dictionary
#missense_dict = main_package.codon_table.iupac_missense_codon_dict(codon_table=args.codon_table)
#missense_dict = main_package.codon_table.iupac_missense_codon_dict(codon_table='Standard')
missense_dict = main_package.codon_table.selected_iupac_codons_dict()

In [3]:
# generate synonymous missense codon dictionary
#main_package.codon_table.iupac_synonymous_codon_dict(codon_table=args.codon_table)
#synonymous_dict = main_package.codon_table.iupac_synonymous_codon_dict(codon_table='Standard')
synonymous_missense_dict = main_package.codon_table.synonymous_iupac_codons_dict()

In [4]:
# generate yeast synonymous codon dictionary (no missense variants)
yeast_synonymous_dict = main_package.codon_table.synonymous_yeast_codons_dict()

In [5]:
# parse .gb file, loop for each matching feature "window"
for window in file:
    # check that window is divisible by 3, codons
    # check for upstream homology and downstream primer space (20bp, 40bp)

    # assign sub-window start index value
    # define which codons will contain synonymous controls at 5% frequency (based on wt)
    # define synonymous codons (based on vector)
    # (create all the codon variants immediately, then define subwindows based on primers)
    
    # begin sub-window loop:
        # homology arm
        # primer design
            # redefine sub-window start index
        # drop-in missense sub-window

IndentationError: expected an indented block (2364269451.py, line 15)

In [6]:
# .gb loop practice: MESSY
wt_input = 'k3l_test.gb'
vector_input = None

# check for vector file
wt_file = SeqIO.read(wt_input, 'genbank')
if vector_input == None:
    vector_input = wt_input
vector_file = SeqIO.read(vector_input, 'genbank')
    
wt_seq = str(wt_file.seq.upper())
vector_seq = str(vector_file.seq.upper())

In [7]:
# get start and stop of gene for codon positions
for feature in wt_file.features:
    if feature.type == 'gene':
        gene_start = feature.location.start.position
        gene_end = feature.location.end.position
        print(gene_start)
        print(gene_end)

1656
1923


In [8]:
# create dataframe
df = pd.DataFrame()

In [9]:
# SANITY CHECKS (make an error check module?)

# check that wt and vector files are equal in length
if len(wt_seq) != len(vector_seq):
    print('Error: WildType and Vector GenBank sequences are not of equal length')
    #break
    
# check for -20 bp homology

# check that the strand is going forward

In [11]:

homo_len = 20 ### args.homo_len
rev_melt_temp = 55 ### args.rev_melt_temp
primer_len = 60 ### args.primer_len
melt_temp = 50 ### args.melt_temp
syn_snp_rate = .05 ### args.syn_snp_rate
stop_rate = .1 ### args.stop_rate

In [13]:
### loop through the window

# setup RNG in the loop, maybe this needs to be moved
rng = np.random.RandomState(42)

# this needs to be fixed (user input? yaml?)
targ_windows = ['window_1', 'window_2', 'window_3']

for feature in wt_file.features:
    if feature.type not in targ_windows:
        continue
    
    # START LOOP: CHANGE THIS
    win_list = []
    homo_list = []
    rev_primer_list = []
    fwd_primer_list = []
    mut_windows_list = []
    mut_name_list = []
    full_window_list = []
    full_name_list = []
    full_primer_list = []
    
    start_index = feature.location.start.position
    window_end = feature.location.end.position
    
    # loop for each mini_window
    mini_win = 1
    while start_index < window_end: # this could be an issue to toggle
        data_dict = {}
        
        # 1. homology arm
        homology_arm = vector_seq[start_index - homo_len:start_index] ### args.homo_len
        data_dict['homology_arm'] = homology_arm
        
        # 2. reverse primer
        reverse_seq = str(Seq(vector_seq[:start_index]).reverse_complement())
        reverse_primer = reverse_seq[:15]
        while mt.Tm_NN(reverse_primer) < rev_melt_temp: ### args.rev_melt_temp
            reverse_primer = reverse_seq[:len(reverse_primer)+1] ### args.rev_melt_temp
        data_dict['reverse_primer'] = reverse_primer
        
        reverse_primer_name = f'rev_{str(feature.type)}-{mini_win}'
        data_dict['reverse_primer_name'] = reverse_primer_name
        
        # 3. forward primer
        primer_end = start_index + (primer_len - homo_len) ### args.primer_len homo_len
        if primer_end > window_end:
            primer_end == window_end
        
        primer_start = primer_end - 15
        forward_primer = vector_seq[primer_start:primer_end]

        while mt.Tm_NN(forward_primer) < melt_temp: ### args.melt_temp
            primer_start -= 1
            forward_primer = vector_seq[primer_start:primer_end]

        # check if the primer is > 28bp
        if len(forward_primer) > (primer_len - homo_len - 12): ### args.primer_len, homo_len
            # fix mut window to 12, make a long primer
            primer_start = start_index + 12
            primer_end = primer_start + 15
            forward_primer = vector_seq[primer_start:primer_end]
            while True:
                forward_primer = vector_seq[primer_start:primer_end]
                if mt.Tm_NN(forward_primer) > melt_temp and forward_primer.upper().count('G') + forward_primer.upper().count('C') > 8: ### args.melt_temp
                    break
                else:
                    primer_end += 1
        
        # even-out the primer length to accomodate codons
        else:
            # add or subtract a bp from the fwd primer to get mut_window in frame
            if (primer_start - start_index)%3 == 2:
                primer_start += 1
                forward_primer = vector_seq[primer_start:primer_end]

            elif (primer_start - start_index)%3 == 1:
                primer_start -= 1
                forward_primer = vector_seq[primer_start:primer_end]

        if primer_start > window_end:
            # make the last fwd primer
            primer_start = window_end
            primer_end = primer_start+15
            forward_primer = vector_seq[primer_start:primer_end]
            while mt.Tm_NN(forward_primer) < melt_temp:
                primer_end += 1
                forward_primer = vector_seq[primer_start:primer_end]
        data_dict['forward_primer'] = forward_primer
        
        # 4. variant window
        mut_len = (primer_start) - start_index
        mut_end = start_index + mut_len

        def codons_list(seq):
            return [seq[i:i+3] for i in range(0, len(seq), 3)]
        
        # removing mis_list and syn_list
        wt_list = codons_list(wt_seq[start_index:mut_end])
        vect_list = codons_list(vector_seq[start_index:mut_end])

        # generate synonymous vector codon list (top 2 codons for yeast)
        synonymous_win = [yeast_synonymous_dict[i] for i in vect_list]
        
        # generate iupac missense codons list (with synonymous codons)  
        doped_codons = []
        for i, wt_codon in enumerate(wt_list):            
            syn_bool = rng.choice([True, False], p=[syn_snp_rate, 1-syn_snp_rate]) ### args.syn_snp_rate
            data_dict['synonymous_included'] = syn_bool
            if syn_bool: 
                doped_codons.append(synonymous_missense_dict[wt_codon])
            else:
                doped_codons.append(missense_dict[wt_codon])

        # generate the mut primer and all info
        for i, iupac_list in enumerate(doped_codons):
            res_pos = int((((start_index-gene_start)/3)+1)+i)
            for iupac_codon in iupac_list:

                codon_sub = wt_list[i] + str(res_pos) + iupac_codon
                variant_win = ''.join(synonymous_win[:i] + [iupac_codon] + synonymous_win[i+1:])
                primer_name = f'{str(feature.type)}-{mini_win}_{codon_sub}'
                primer = homology_arm + variant_win + forward_primer
                
                dict_keys = ['name','codon_sub','wt','pos','iupac']
                dict_values = [primer_name, codon_sub, wt_list[i], res_pos, iupac_codon]
                for (key,value) in zip(dict_keys,dict_values):
                    data_dict[key] = value

                # append values to dataframe
                df = df.append(data_dict, ignore_index=True)
     
        # reset the start index for the next mini-window
        start_index = primer_start
        mini_win += 1  

dict_keys = ['name','codon_sub','wt','pos','iupac','syn_bool','homo','variant_win','primer','rev_primer_name','rev_primer']        
        
# organize data into .tsv 

# to add
"""
'primer':,
'primer_tm':,
'primer_gc':,
'primer_len':,
'rev_temp':,
'rev_gc':,
'rev_primer_len':, 
"""
    

"\n'primer':,\n'primer_tm':,\n'primer_gc':,\n'primer_len':,\n'rev_temp':,\n'rev_gc':,\n'rev_primer_len':, \n"

Seq('X')

In [14]:
df

,homology_arm,reverse_primer,reverse_primer_name,forward_primer,synonymous_included,name,codon_sub,wt,pos,iupac
0,TTTGTTATTCGTTGCCCAAT,ATTGGGCAACGAATAACAAAATGCA,rev_window_1-1,GGCAGAGTATACGAGAAGGATT,0.0,window_1-1_GCG11HCG,GCG11HCG,GCG,11.0,HCG
1,TTTGTTATTCGTTGCCCAAT,ATTGGGCAACGAATAACAAAATGCA,rev_window_1-1,GGCAGAGTATACGAGAAGGATT,0.0,window_1-1_GCG11GDG,GCG11GDG,GCG,11.0,GDG
2,TTTGTTATTCGTTGCCCAAT,ATTGGGCAACGAATAACAAAATGCA,rev_window_1-1,GGCAGAGTATACGAGAAGGATT,0.0,window_1-1_GGT12HGT,GGT12HGT,GGT,12.0,HGT
3,TTTGTTATTCGTTGCCCAAT,ATTGGGCAACGAATAACAAAATGCA,rev_window_1-1,GGCAGAGTATACGAGAAGGATT,0.0,window_1-1_GGT12GHT,GGT12GHT,GGT,12.0,GHT
4,TTTGTTATTCGTTGCCCAAT,ATTGGGCAACGAATAACAAAATGCA,rev_window_1-1,GGCAGAGTATACGAGAAGGATT,0.0,window_1-1_GAT13HAT,GAT13HAT,GAT,13.0,HAT
...,...,...,...,...,...,...,...,...,...,...
92,TCAATTACAAAAGGATGTGT,ACACATCCTTTTGTAATTGACATCTATATATCCT,rev_window_3-4,TAGTAGCTGTCGAGTCGC,0.0,window_3-4_CAT87CBT,CAT87CBT,CAT,87.0,CBT
93,TCAATTACAAAAGGATGTGT,ACACATCCTTTTGTAATTGACATCTATATATCCT,rev_window_3-4,TAGTAGCTGTCGAGTCGC,0.0,window_3-4_CAT87CAA,CAT87CAA,CAT,87.0,CAA
94,TCAATTACAAAAGGATGTGT,ACACATCCTTTTGTAATTGACATCTATATATCCT,rev_window_3-4,TAGTAGCTGTCGAGTCGC,0.0,window_3-4_CAA88DAA,CAA88DAA,CAA,88.0,DAA
95,TCAATTACAAAAGGATGTGT,ACACATCCTTTTGTAATTGACATCTATATATCCT,rev_window_3-4,TAGTAGCTGTCGAGTCGC,0.0,window_3-4_CAA88CBA,CAA88CBA,CAA,88.0,CBA


In [47]:
window_end

1920

In [14]:
(start_index-gene_start)/3


70.0

In [24]:
Seq('ATTGGGCAACGAATA').reverse_complement()

Seq('TATTCGTTGCCCAAT')

In [25]:
vector_seq

'CGAACGCCAGCAAGACGTAGCCCAGCGCGTCGGCCGCCATGCCGGCGATAATGGCCTGCTTCTCGCCGAAACGTTTGGTGGCGGGACCAGTGACGAAGGCTTGAGCGAGGGCGTGCAAGATTCCGAATACCGCAAGCGACAGGCCGATCATCGTCGCGCTCCAGCGAAAGCGGTCCTCGCCGAAAATGACCCAGAGCGCTGCCGGCACCTGTCCTACGAGTTGCATGATAAAGAAGACAGTCATAAGTGCGGCGACGATAGTCATGCCCCGCGCCCACCGGAAGGAGCTGACTGGGTTGAAGGCTCTCAAGGGCATCGGTCGACGCTCTCCCTTATGCGACTCCTGCATTAGGAAGCAGCCCAGTAGTAGGTTGAGGCCGTTGAGCACCGCCGCCGCAAGGAATGGTGCATGCAAGGAGATGGCGCCCAACAGTCCCCCGGCCACGGGGCCTGCCACCATACCCACGCCGAAACAAGCGCTCATGAGCCCGAAGTGGCGAGCCCGATCTTCCCCATCGGTGATGTCGGCGATATAGGCGCCAGCAACCGCACCTGTGGCGCCGGTGATGCCGGCCACGATGCGTCCGGCGTAGAGGATCCACAGGACGGGTGTGGTCGCCATGATCGCGTAGTCGATAGTGGCTCCAAGTAGCGAAGCGAGCAGGACTGGGCGGCGGCCAAAGCGGTCGGACAGTGCTCCGAGAACGGGTGCGCATAGAAATTGCATCAACGCATATAGCGCTAGCAGCACGCCATAGTGACTGGCGATGCTGTCGGAATGGACGATATCCCGCAAGAGGCCCGGCAGTACCGGCATAACCAAGCCTATGCCTACAGCATCCAGGGTGACGGTGCCGAGGATGACGATGAGCGCATTGTTAGATTTCATACACGGTGCCTGACTGCGTTAGCAATTTAACTGTGATAAACTACCGCATTAAAGCTTATCGATGAGCTCCTTATGCGGATCTGTAGCAGCTGTCATTATCAATACTGCCA

In [ ]:
win_list
homo_list
rev_primer_list
fwd_primer_list
mut_windows_list
mut_name_list
full_window_list
full_name_list
full_primer_list

In [ ]:
# need to double-check the conditional logic on the indices

In [23]:
wt_seq[start_index:window_end]

'GATTATACAAAAGGATATATAGATGTCAATTACAAAAGGATGTGTAGACATCAA'

In [14]:
feature.location.start.position

1923

In [ ]:
# SKETCH: aggregate into dataframe